In [1]:
import tt
import numpy as np
import time

In [2]:
def orthogonalize(coresX):
    def reshape(a, sz):
        return np.reshape(a, sz, order="F")
    for dim in range(len(coresX) - 1):
        cc = coresX[dim]
        r1, n, r2 = cc.shape
        cc, rr = np.linalg.qr(reshape(cc, (-1, r2)))
        r2 = cc.shape[1]
        coresX[dim] = reshape(cc, (r1, n, r2))
        coresX[dim+1] = np.tensordot(rr, coresX[dim+1], 1)
    return coresX

In [2]:
def create_inverse_index_sums_tensorized(n, d, QTT_d):
    nats = tt.vector.to_list(tt.xfun(n, QTT_d))
    base = tt.vector.to_list(tt.ones(n, QTT_d * d))
    X = []
    for i in range(d):
        tmp = base.copy()
        for j in range(QTT_d):
            tmp[i * QTT_d + j] = nats[j].copy()
        X.append(tt.vector.from_list(tmp))
    c = tt.multifuncrs2(X, lambda x: 1 / (1 + np.sum(x, axis = 1)), eps=1E-6, verb=0)
    return c

In [11]:
create_inverse_index_sums_tensorized(2, 4, 15)

This is a 60-dimensional tensor 
r(0)=1, n(0)=2 
r(1)=2, n(1)=2 
r(2)=4, n(2)=2 
r(3)=8, n(3)=2 
r(4)=11, n(4)=2 
r(5)=12, n(5)=2 
r(6)=13, n(6)=2 
r(7)=15, n(7)=2 
r(8)=16, n(8)=2 
r(9)=17, n(9)=2 
r(10)=19, n(10)=2 
r(11)=20, n(11)=2 
r(12)=21, n(12)=2 
r(13)=22, n(13)=2 
r(14)=24, n(14)=2 
r(15)=23, n(15)=2 
r(16)=23, n(16)=2 
r(17)=23, n(17)=2 
r(18)=23, n(18)=2 
r(19)=23, n(19)=2 
r(20)=23, n(20)=2 
r(21)=23, n(21)=2 
r(22)=22, n(22)=2 
r(23)=22, n(23)=2 
r(24)=22, n(24)=2 
r(25)=22, n(25)=2 
r(26)=23, n(26)=2 
r(27)=23, n(27)=2 
r(28)=23, n(28)=2 
r(29)=22, n(29)=2 
r(30)=21, n(30)=2 
r(31)=21, n(31)=2 
r(32)=21, n(32)=2 
r(33)=21, n(33)=2 
r(34)=21, n(34)=2 
r(35)=21, n(35)=2 
r(36)=21, n(36)=2 
r(37)=21, n(37)=2 
r(38)=21, n(38)=2 
r(39)=21, n(39)=2 
r(40)=21, n(40)=2 
r(41)=21, n(41)=2 
r(42)=21, n(42)=2 
r(43)=21, n(43)=2 
r(44)=21, n(44)=2 
r(45)=20, n(45)=2 
r(46)=20, n(46)=2 
r(47)=20, n(47)=2 
r(48)=19, n(48)=2 
r(49)=18, n(49)=2 
r(50)=18, n(50)=2 
r(51)=18, n(51)=2 
r(5

In [3]:
def average_rank(T):
    K = tt.vector.to_list(T)
    sum0 = 0
    size = 0
    for i in range(len(K) - 1):
        sum0 += K[i].shape[2]
        size += 1
    return sum0 / size

def maximal_rank(T):
    K = tt.vector.to_list(T)
    maxi = 0
    for i in range(len(K) - 1):
        maxi = max(K[i].shape[2], maxi)
    return maxi

In [16]:
C = create_inverse_index_sums_tensorized(2, 4, 15)
B = create_inverse_index_sums_tensorized(2, 3, 20)
A = create_inverse_index_sums_tensorized(2, 2, 30)

print(maximal_rank(A))
print()
print(maximal_rank(B))
print()
print(maximal_rank(C))

41

28

24


In [28]:
from bilinear_package.src.contraction import partialContractionsRLKronecker

def count_scalar_product(A, B, R, norm):
    sc_prod = partialContractionsRLKronecker(A, B, R)
    norm_R = tt.vector.norm(tt.vector.from_list(R)) ** 2
    return np.sqrt(np.abs(norm_R + norm ** 2 - 2 * sc_prod[0][0, 0, 0])) / norm

In [29]:
from bilinear_package.src.hadamard_product import approximateHadamardProduct
import pandas as pd

def experiment():
    C = create_inverse_index_sums_tensorized(2, 4, 15)
    B = create_inverse_index_sums_tensorized(2, 3, 20)
    A = create_inverse_index_sums_tensorized(2, 2, 30)
    k = 0
    df_time = pd.DataFrame()
    df_precision = pd.DataFrame()
    for T in [C, B, A]:
        X = T * T
        norm = tt.vector.norm(X)
        r = maximal_rank(T)
        for rank in range(10, 47, 4):
            ranks = [rank for _ in range(len(tt.vector.to_list(T)) - 1)]
            time1 = time.time()
            answer1 = approximateHadamardProduct(tt.vector.to_list(T), tt.vector.to_list(T), desired_ranks = ranks, seed= 21 + rank * 3 + k )
            U = tt.vector.to_list(T)
            time2 = time.time()
            answer2 = tt.vector.to_list(X.round(0, rank))
            time3 = time.time()
            answer3 = tt.vector.to_list(tt.multifuncrs2([T, T], lambda x : x[:, 0] * x[:, 1], eps=1e-6, verb = 0))
            time4 = time.time()
            # precision1 = count_scalar_product(U, U, answer1, norm)
            # precision2 = count_scalar_product(U, U, answer2, norm)
            # precision3 = count_scalar_product(U, U, answer3, norm)
            precision1 = tt.vector.norm(X - tt.vector.from_list(answer1)) / norm
            precision2 = tt.vector.norm(X - tt.vector.from_list(answer2)) / norm
            precision3 = tt.vector.norm(X - tt.vector.from_list(answer3)) / norm
            df_time[f"{r}_{rank}"] = [time2 - time1, time3 - time2, time4 - time3]
            df_precision[f"{r}_{rank}"] = [precision1, precision2, precision3]
            df_time.to_csv("experiments_results/experiment3/time.csv")
            df_precision.to_csv("experiments_results/experiment3/presicion.csv")
        k += 1

In [52]:
experiment()

In [4]:
from bilinear_package.src.hadamard_product import approximateHadamardProduct
import pandas as pd

def experiment_oversampling():
    A = create_inverse_index_sums_tensorized(2, 2, 30)
    k = 0
    for T in [A]:
        U = tt.vector.to_list(T)
        X = T * T
        norm = tt.vector.norm(X)
        r = maximal_rank(T)
        by_rank = 60
        answer = []
        df = pd.DataFrame()
        answer = [0 for _ in range(5)]
        for rank in range(5, 61):
            ranks = [rank for _ in range(len(tt.vector.to_list(T)) - 1)]
            answer.append(approximateHadamardProduct(tt.vector.to_list(T), tt.vector.to_list(T), desired_ranks = ranks, seed = 21 + rank * 3 + k))
        for base_rank in range(5, 51, 2):
            column = []
            for oversampling in [0, 1, 2, 4, 6, 9]:
                answer1 = tt.vector.to_list(tt.vector.from_list(answer[base_rank + oversampling]).round(0, base_rank))
                precision1 = tt.vector.norm(X - tt.vector.from_list(answer1)) / norm
                column.append(precision1)
            df[base_rank] = column
            df.to_csv(f"experiments_results/experiment3/oversampling.csv")
        k += 1

In [5]:
experiment_oversampling()